# 毒キノコの判定 SVM

## SVMとは何か

### どのようなときに使うのか

分類にも回帰にも使える手法。

### どのような仕組みなのか

分類時は、マージンを最大化する境界線の引き方をする。
回帰は、マージンの中にうまく含むようにモデルを表す。

### どういった利点欠点があるのか

利点は、汎化能力が高く、最も良い精度を持つ手法のひとつ。
欠点は使用するカーネルによって、計算時間がかかること。

## データを取得

https://archive.ics.uci.edu/ml/datasets/Mushroom

In [5]:
# データを読み込むコードを記述
import pandas as pd
data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data", header=None)
features = ['Poisonous', 'cap-shape', 'cap-surface', 'cap-color', 'bruises?', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size',
              'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring',
              'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population', 
              'habitat']
data.columns = features

## データ確認

In [6]:
# df.などにより取得したデータを確認
data.head(10)

,Poisonous,cap-shape,cap-surface,cap-color,bruises?,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
5,e,x,y,y,t,a,f,c,b,n,...,s,w,w,p,w,o,p,k,n,g
6,e,b,s,w,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,n,m
7,e,b,y,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,s,m
8,p,x,y,w,t,p,f,c,n,p,...,s,w,w,p,w,o,p,k,v,g
9,e,b,s,y,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,s,m


## データの変換

In [7]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for feature in features:
    data[feature] = le.fit_transform(data[feature])
data.head(10)

,Poisonous,cap-shape,cap-surface,cap-color,bruises?,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1
5,0,5,3,9,1,0,1,0,0,5,...,2,7,7,0,2,1,4,2,2,1
6,0,0,2,8,1,0,1,0,0,2,...,2,7,7,0,2,1,4,2,2,3
7,0,0,3,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,3,3
8,1,5,3,8,1,6,1,0,1,7,...,2,7,7,0,2,1,4,2,4,1
9,0,0,2,9,1,0,1,0,0,2,...,2,7,7,0,2,1,4,2,3,3


## データセットの分割

In [8]:
# testデータとtrainデータに分割
# データセットを分割するコードの記述

from sklearn.model_selection import train_test_split

data_train, data_test=train_test_split(data,test_size=0.2,random_state=0)

X_train = data_train.drop(['Poisonous'], axis=1)
y_train = data_train['Poisonous']

X_test = data_test.drop(['Poisonous'], axis=1)
y_test = data_test['Poisonous']

## 標準化

In [9]:
# 標準化を行うコードを記述
from sklearn.preprocessing import StandardScaler

std_scl = StandardScaler()
std_scl.fit(X_train)
X_train = std_scl.transform(X_train)
X_test = std_scl.transform(X_test)

### なぜSVMを行う際に標準化が必要なのか

SVMは特徴量のスケールが揃っていないと上手く学習できないアルゴリズムであるため、
標準化(平均0、分散1となるように変換)することが必要となる。

## カーネルとは何か

カーネルとはふたつのベクトルの内積を高次元へ移すこと

### どのように使い分けるのか

- linear（線形カーネル）は他のカーネルと比べて、ちょっとしたデータセットでも体感できるほどに処理が遅い
- poly（多項式カーネル）はハイパーパラメータの数が4つで、グリッドサーチの負担が大きい
- rbf（RBFカーネル）はデフォルト

## コストペナルティCとは何か

SVMで分類されたモデルでどれだけ誤分類されているかの度合い

## ハイパーパラメータ

In [10]:
# グリッドサーチのコードを記述

from sklearn.grid_search import GridSearchCV

parameters_list = [{'C': [0.1,1, 10, 100], 'kernel': ['rbf'], 'gamma': [0.1, 0.01, 0.001]}]

from sklearn.svm import SVC
clf_rbf = GridSearchCV(SVC(), parameters_list, cv=5)
clf_rbf.fit(X_train, y_train) 
clf_rbf.grid_scores_

C:\Users\kakiuchi\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\kakiuchi\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


[mean: 0.99862, std: 0.00058, params: {'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'},
 mean: 0.95184, std: 0.00308, params: {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'},
 mean: 0.88721, std: 0.00406, params: {'C': 0.1, 'gamma': 0.001, 'kernel': 'rbf'},
 mean: 1.00000, std: 0.00000, params: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'},
 mean: 0.98615, std: 0.00266, params: {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'},
 mean: 0.94768, std: 0.00374, params: {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'},
 mean: 1.00000, std: 0.00000, params: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'},
 mean: 1.00000, std: 0.00000, params: {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'},
 mean: 0.95476, std: 0.00431, params: {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'},
 mean: 1.00000, std: 0.00000, params: {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'},
 mean: 1.00000, std: 0.00000, params: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'},
 mean: 0.98892, std: 0.00246, params: {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}]

## SVMの学習およびテスト

In [11]:
# SVMの学習およびテストをするコードを記述

svm = SVC(kernel='rbf', C=100, gamma=0.01)
svm = svm.fit(X_train, y_train)
y_svm = svm.predict(X_test)

from sklearn.metrics import classification_report
test_report = classification_report(y_test, y_svm)

y_svm = svm.fit(X_train, y_train).predict(X_train)
train_report = classification_report(y_train,y_svm)

print('test_report')
print(test_report)
print('train_report')
print(train_report)

test_report
             precision    recall  f1-score   support

          0       1.00      1.00      1.00       852
          1       1.00      1.00      1.00       773

avg / total       1.00      1.00      1.00      1625

train_report
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      3356
          1       1.00      1.00      1.00      3143

avg / total       1.00      1.00      1.00      6499

